# Unmapped Metabase

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Catalog/Unmapped/data'
STORE = 'alianza'

## Import

In [2]:
import sys
sys.path.append('/Users/efraflores/Desktop/hub/cornershop/venv/lib/python3.9/site-packages')

In [3]:
import os
import time
import numpy as np
import pandas as pd
from IPython.lib.display import Audio

prod = pd.read_csv(os.path.join(BASE_DIR,f'catalog_prod_{STORE}.csv'))
supply = pd.read_csv(os.path.join(BASE_DIR,f'catalog_supply_{STORE}.csv'))

display(prod.sample())
display(supply.sample())

,store_id,catalog_product_id,branch,productbranch_id,price
93203,98,218648,935,8226199,36.0


,sku_source,supply_product_id,catalog_product_id,name,barcode,branch,price_mode,stock,min_last_visit
93396,mx-bodegasalianza-csv,14140837,NaN,Bdy. torres c/2 vasos,8.410114e+12,941.0,500.0,1,2021-02-18T17:06:43.601919Z


## Functions

### Timing and tone

In [4]:
start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

### Batches

In [5]:
def batches(base_dir,store,data,size=10000):
    df = data.copy()
    n = 0
    for i in range(int(np.ceil(len(df)/size))):
        batch = df.iloc[n:n+size-1,:]
        batch.to_excel(os.path.join(base_dir,f'bulk_{store}_{str(i+1).zfill(2)}.xlsx'),index=False)
        n += size
    print(f'The file with {len(df)} rows has been split up into {i+1} batches of {size} rows each')

### Summary

In [6]:
def summary():
    print(f"Para la tienda {STORE.title()}, los productbranch se distribuyen:\
    \n-Catálogo\t{len(prod)}\
    \n-Supply\t\t{len(supply)}\
    \n-Sin mapear\t{len(unmapped)+len(brands)}\t({'{:.1%}'.format((len(unmapped)+len(brands))/len(full))})\
    \n -Brands\t{len(branch_brands)}\
    \n -Sin órdenes\t{len(df[df['orders']==0])}\
    \n -Buen perform\t{len(good_perf)}\t(FR {'{:.1%}'.format(good_perf['fr'].mean())})\
    \n -Mal perform\t{len(bad_fr)}\t(FR {'{:.1%}'.format(bad_fr['fr'].mean())})\
    \n -Markup +-3%\t{len(mk)}\t(MK {int(mk['mk_tot'].sum())}, {'{:.1%}'.format(mk['mk_tot'].sum()/mk['sales'].sum())})\
    \n\n-Por desactivar\t{len(bad_perf)}\t({'{:.1%}'.format(len(bad_perf)/len(prod))})\
    \n -Productos \t{len(bad_perf.groupby('catalog_product_id')['store_id'].count())}\
    \n -Órdenes\t{int(bad_perf['orders'].sum())}\t({'{:.1%}'.format(bad_perf['orders'].sum()/total['orders'].sum())})\
    \n -Ventas $ \t{int(bad_perf['sales'].sum())}\t({'{:.1%}'.format(bad_perf['sales'].sum()/total['sales'].sum())})")

## Unmapped from prod

In [7]:
full = prod.merge(supply,how='left',on=['branch','catalog_product_id'])
full.isnull().mean()

store_id              0.000000
catalog_product_id    0.000000
branch                0.000000
productbranch_id      0.000000
price                 0.000000
sku_source            0.596777
supply_product_id     0.596777
name                  0.596777
barcode               0.596777
price_mode            0.596777
stock                 0.596777
min_last_visit        0.596777
dtype: float64

In [8]:
unmapped = full[full['supply_product_id'].isnull()].dropna(axis=1)
unmapped.to_csv(os.path.join(BASE_DIR,f'check_brands_{STORE}.csv'),index=False)

## Brands

Check with @Alan Sánchez Márquez if any product_id was created by Brands

In [9]:
brands = pd.read_csv(os.path.join(BASE_DIR,f'checked_{STORE}.csv'))
brands

,catalog_product_id,match
0,3906875,1
1,1116980,1
2,1154000,1
3,368209,1
4,383363,1
5,384378,1
6,1245642,1
7,1014261,1
8,323692,1


In [10]:
branch_brands = unmapped[unmapped['catalog_product_id'].isin(brands['catalog_product_id'])]
unmapped = unmapped[~unmapped['catalog_product_id'].isin(brands['catalog_product_id'])].copy()
','.join(set(unmapped['catalog_product_id'].dropna().astype(int).astype(str)))

'360778,2255893,129859,383588,174089,96865,383520,141514,1155662,357349,281975,353436,151314,383333,355849,383530,168683,280153,345551,591043,383494,1126262,383635,171286,383597,1993944,129828,353032,319735,594240,139185,383632,383405,383293,604813,140166,383353,138258,592705,2161160,2442313,328885,383335,323243,357658,383507,383546,383590,383610,383591,383580,220847,140638,1230746,383498,383263,140346,170562,383282,324074,383390,591911,383585,383558,383426,141515,320402,328175,356222,383272,323181,383294,367500,601009,1109195,379985,137605,338578,314560,137348,592961,313187,346098,137632,592630,320422,341897,383318,140550,324090,367819,388156,343777,383281,343952,367441,312416,383249,353651,316255,356499,383447,138468,383509,1156222,589676,376684,383436,383469,357035,226008,341893,328267,383277,323736,1006858,1217230,367609,346023,1188429,357712,383283,594422,383310,383264,356041,383526,383455,360845,369727,383619,317231,383645,321222,375796,444769,595373,383560,413366,383247,341903,3

## Orders from unmapped

In [11]:
orders = pd.read_csv(os.path.join(BASE_DIR,f'orders_{STORE}.csv'))
orders.head()

,level,date,catalog_product_id,branch,productbranch_id,min_branch_price,avg_branch_price,max_branch_price,min_shopper_price,avg_shopper_price,max_shopper_price,requested,found,orders,sales
0,TOTAL,2021-06-11,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,115.0,100.0,60,54349.99
1,TOTAL,2021-06-12,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,87.0,52,49309.74
2,TOTAL,2021-06-10,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,83.0,69.0,39,36009.10
3,TOTAL,2021-06-13,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,17.0,10,9242.37
4,product_branch,2021-06-11,336122,7651,38195757,150.0,150.0,150.0,150.0,150.0,150.0,3.0,3.0,2,450.00


In [12]:
total = orders[orders['level']=='TOTAL'].iloc[:,-4:].copy()
total['fr'] = total['found']/total['requested']
total['avg_items'] = total['requested']/total['orders']
total

,requested,found,orders,sales,fr,avg_items
0,115.0,100.0,60,54349.99,0.869565,1.916667
1,104.0,87.0,52,49309.74,0.836538,2.000000
2,83.0,69.0,39,36009.10,0.831325,2.128205
3,22.0,17.0,10,9242.37,0.772727,2.200000


### Total

In [13]:
df = unmapped.merge(orders.iloc[:,3:],on='productbranch_id',how='left').fillna(0)
df = df[~df['catalog_product_id'].isin(brands['catalog_product_id'])].copy()
df = df.sort_values('orders',0,0).reset_index(drop=True)
df['fr'] = (df['found']/(df['requested']+1e-10)).apply(lambda x: min(1,x))
print(df['fr'].describe())
df.head()

count    77326.000000
mean         0.001145
std          0.033764
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: fr, dtype: float64


,store_id,catalog_product_id,branch_x,productbranch_id,price,branch_y,min_branch_price,avg_branch_price,max_branch_price,min_shopper_price,avg_shopper_price,max_shopper_price,requested,found,orders,sales,fr
0,98,352963,916,8239069,635.0,916.0,635.0,635.0,635.0,610.0,610.0,610.0,2.0,2.0,2.0,1270.0,1.0
1,98,1993298,7628,45769157,295.0,7628.0,295.0,295.0,295.0,325.0,325.0,325.0,2.0,1.0,2.0,295.0,0.5
2,98,383601,971,8268497,164.0,971.0,164.0,164.0,164.0,164.0,164.0,164.0,1.0,1.0,1.0,164.0,1.0
3,98,140711,920,8235607,152.0,920.0,152.0,152.0,152.0,160.0,160.0,160.0,2.0,2.0,1.0,304.0,1.0
4,98,383347,950,8258469,180.0,950.0,180.0,180.0,180.0,180.0,180.0,180.0,1.0,1.0,1.0,180.0,1.0


### Bad performance

BULK TOOL

<https://cornershopapp.com/catalog/dashboard/bulk_update/product_branches/>

In [14]:
df['avg_price'] = df['sales'] / df['found']
df['mk_un'] = df['avg_price'] - df['price']
df['mk_p'] = df['mk_un'] / df['avg_price']
df['mk_tot'] = df['sales'] - df['price']*df['found']
bad_fr = df[(df['fr']<=total['fr'][0]*.9)
            #&(df['orders']>0)
           ].copy()
mk = df[(df['mk_p']<-.03)|(df['mk_p']>.03)].copy()
bad_perf = bad_fr.append(mk)
print(bad_perf['fr'].describe())
bad_perf.head()

count    77239.000000
mean         0.000019
std          0.004023
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: fr, dtype: float64


,store_id,catalog_product_id,branch_x,productbranch_id,price,branch_y,min_branch_price,avg_branch_price,max_branch_price,min_shopper_price,...,max_shopper_price,requested,found,orders,sales,fr,avg_price,mk_un,mk_p,mk_tot
1,98,1993298,7628,45769157,295.0,7628.0,295.0,295.0,295.0,325.0,...,325.0,2.0,1.0,2.0,295.0,0.5,295.0,0.0,0.0,0.0
5,98,2467701,7668,67376894,142.0,7668.0,142.0,142.0,142.0,0.0,...,0.0,2.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,0.0
8,98,137656,927,8231477,165.0,927.0,165.0,165.0,165.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,0.0
12,98,386797,7651,38194859,221.0,7651.0,221.0,221.0,221.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,0.0
17,98,383521,930,8262912,445.0,930.0,445.0,445.0,445.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,0.0


### Good performance

Ask the KAM to talk to the partner in order to activate that product

In [15]:
good_perf = df[(~df['productbranch_id'].isin(bad_perf['productbranch_id']))&
               (df['orders']>0)].copy()
good_perf.to_csv(os.path.join(BASE_DIR,f'good_perf_{STORE}.csv'),index=False)
print(good_perf['fr'].describe())
good_perf.head()

count    8.700000e+01
mean     1.000000e+00
std      2.668238e-11
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: fr, dtype: float64


,store_id,catalog_product_id,branch_x,productbranch_id,price,branch_y,min_branch_price,avg_branch_price,max_branch_price,min_shopper_price,...,max_shopper_price,requested,found,orders,sales,fr,avg_price,mk_un,mk_p,mk_tot
0,98,352963,916,8239069,635.0,916.0,635.0,635.0,635.0,610.0,...,610.0,2.0,2.0,2.0,1270.0,1.0,635.0,0.0,0.0,0.0
2,98,383601,971,8268497,164.0,971.0,164.0,164.0,164.0,164.0,...,164.0,1.0,1.0,1.0,164.0,1.0,164.0,0.0,0.0,0.0
3,98,140711,920,8235607,152.0,920.0,152.0,152.0,152.0,160.0,...,160.0,2.0,2.0,1.0,304.0,1.0,152.0,0.0,0.0,0.0
4,98,383347,950,8258469,180.0,950.0,180.0,180.0,180.0,180.0,...,180.0,1.0,1.0,1.0,180.0,1.0,180.0,0.0,0.0,0.0
6,98,345459,935,8226420,412.0,935.0,412.0,412.0,412.0,412.0,...,412.0,1.0,1.0,1.0,412.0,1.0,412.0,0.0,0.0,0.0


## Bulk

In [31]:
#bad_perf#.to_csv(os.path.join(BASE_DIR,f'bad_performance_{STORE}.csv'),index=False)

In [16]:
export = bad_perf[['store_id','productbranch_id']].copy()
export.rename(columns={'productbranch_id':'product_branch_id'},inplace=True)
export[['price','availability','schedule']] = ''
export.head()

,store_id,product_branch_id,price,availability,schedule
1,98,45769157,,,
5,98,67376894,,,
8,98,8231477,,,
12,98,38194859,,,
17,98,8262912,,,


In [17]:
batches(BASE_DIR,STORE,export)

The file with 77239 rows has been split up into 8 batches of 10000 rows each


## Summary

In [18]:
time_exp(time.time()-start)
tono()

0 minutos con 7.75 segundos


In [19]:
summary()

Para la tienda Alianza, los productbranch se distribuyen:    
-Catálogo	128740    
-Supply		131221    
-Sin mapear	77334	(59.3%)    
 -Brands	450    
 -Sin órdenes	77200    
 -Buen perform	87	(FR 100.0%)    
 -Mal perform	77238	(FR 0.0%)    
 -Markup +-3%	1	(MK 15, 5.4%)    

-Por desactivar	77239	(60.0%)    
 -Productos 	1189    
 -Órdenes	40	(24.8%)    
 -Ventas $ 	575	(0.4%)
